In [37]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import csv

In [38]:
path_overall = 'C:\\Users\\arvra\\Documents\\UVa files\\Classes\\Fall_18\\Capstone Project\\books\\books'

### Function to extract Event related BESS tag from the XML file

In [55]:
#### Function to extract feature from each of the description types

def BESS_extract_function(each_member, event_description, Model_Data_frame):
    ##### PERSONA DESCRIPTION ############
    ## Checking if the tag matches the text in event_description
            
    ##Getting all the sub tags under events
    sub_members = each_member.getchildren()

    ## textUnitRangeReference
    ## Creating a variable for para_number to store the paragraph number
    para_number = None


    ### Looping through each of the sub tags under Events
    for each in sub_members:

        if(each.text == None):
            continue

        ### Check if we have start and end tag for each of the sub tag ##

        ### Absence of Start and End subtag under the tags
        ### i.e. the tag corresponds to only one paragraph
        if(len(each.getchildren()) == 0):
            #print(each.tag)
            #print(each.text)

            if(each.tag == 'textUnitReference'):
                para_number = each.text.strip()
            else:

                #print(para_number)
                if(para_number == None):
                    break
                    
                if(each.text == None):
                    content_each_text = ""
                else:
                    content_each_text = each.text.strip()


                Persona_Data_temp = pd.DataFrame({'para no' : [para_number],
                                                  'Type' : [event_description],\
                                                  'Event': each.tag ,\
                                                  'Content': [content_each_text]})
                
                Model_Data_frame = pd.concat([Model_Data_frame,Persona_Data_temp], axis = 0)
        
        #print(Event_Data_temp)
        ### Presence of Start and End subtag under the tags
        ### i.e. the tag corresponds to more than one paragraph

        if(len(each.getchildren()) != 0 and each.tag == 'textUnitRangeReference'):
            start_para = int(each.getchildren()[0].text)
            end_para = int(each.getchildren()[1].text)+1
            
            ##
            for group_para in range(start_para,end_para):
                counts_after_index = 0
                for each_group_para in sub_members[1:len(sub_members)]:

                    #print(group_para)
                    
                    if(each_group_para.text == None):
                        content_text = ""
                    else:
                        content_text = each_group_para.text.strip()
                    
                    Persona_Data_temp = pd.DataFrame({'para no' : [group_para],\
                                                    'Type' : [event_description],\
                                                    'Event': each_group_para.tag,\
                                                    'Content': [content_text]})
                    Model_Data_frame = pd.concat([Model_Data_frame,Persona_Data_temp], axis = 0)
                    #print(Event_Data_temp)
                    counts_after_index+= 1
            #print(Persona_Data_frame)

            for each in range(counts_after_index):
                continue
                
    return(Model_Data_frame)

### Primary function to parse through XML files
#### BESS_extract_function function is used within this function to call each category like 'Events', 'PersonaDescription' separately

In [56]:
def bess_to_df(bess_file_path):
    
    #### Checking if it is a valid file before using xml parser
    valid_file = 0
    for each in open(bess_file_path,'r'):
        
        if("BESS" in each):
            #print(each)
            valid_file = 1
            break
    
    ## If there is no valid text, we return without any processing
    if(valid_file == 0):
        return(None)
    
    bess_file_read = bess_file_path
    tree = ET.parse(bess_file_read)
    root = tree.getroot()
    
    ### Checking if we have any information at all
    
    if(root.find('textAnalyzed') == None):
        return(None)
    
    ########################## STARTING THE ACTUAL PARSING #########################################
    #################### Getting Meta data infromation by parsing XML ##############################
    
    textAnalyzed = root.find('textAnalyzed')
    c_id = textAnalyzed.find('collectionID').text
    b_id = textAnalyzed.find('biographyID').text
    uri = textAnalyzed.find('URI').text
    
    ## Setting Author to Unknown if we do not have an author
    if( textAnalyzed.find('author') == None):
        author = "Unknown"
    else:
        author = textAnalyzed.find('author').text
    
    ## Setting the Title to Unknown if we do not have a title
    if( textAnalyzed.find('title') == None):
        title = "Unknown"
    else:
        title = textAnalyzed.find('title').text
    
    persona_name = textAnalyzed.find('personaName').text
    
    ## Creating Dataframe for Meta data and BESS tags
    Model_data_frame = pd.DataFrame({'para no' : [],'Type':[], 'Event' : [],'Content': []})
    topos_Data_frame = discourse_Data_frame = sol_Data_frame = Persona_Data_frame = Event_Data_frame = Model_data_frame
    
    Meta_data_df = pd.DataFrame({'collectionID': [c_id],'biographyID': [b_id],
                                 'URI': [uri],'author': [author],'title': [title],
                                 'personaName': [persona_name]})
    
    
    #################### Getting BESS tag infromation by parsing XML ##########################
    
    for each_member in root.find('analysis'):
        
        ## Checking if the tag is Event (since we are parsing for event here)
        if(each_member.tag == 'event'):
            Event_Data_frame = BESS_extract_function(each_member,"Event",Event_Data_frame)
        
        if(each_member.tag == 'personaDescription'):
            Persona_Data_frame = BESS_extract_function(each_member,"personaDescription",Persona_Data_frame)
        
        if(each_member.tag == 'stageOfLife'):
            sol_Data_frame = BESS_extract_function(each_member,"stageOfLife",sol_Data_frame)
        
        if(each_member.tag == 'discourse'):
            discourse_Data_frame = BESS_extract_function(each_member,"discourse",discourse_Data_frame)
        
        if(each_member.tag == 'topos'):
            topos_Data_frame = BESS_extract_function(each_member,"topos",topos_Data_frame)
        
        if(each_member.tag not in \
           ['event','personaDescription','stageOfLife','discourse','topos','personaType']):
            print(each_member.tag)
            
    ### Events concat
    Events = pd.concat([Event_Data_frame,Meta_data_df], axis = 1)
    Events.reset_index(drop = True,inplace = True)
    
    ### Persona concat
    Persona = pd.concat([Persona_Data_frame,Meta_data_df], axis = 1)
    Persona.reset_index(drop = True,inplace = True)
    
    ## Stage of Life
    
    Stageoflife = pd.concat([sol_Data_frame,Meta_data_df], axis = 1)
    Stageoflife.reset_index(drop = True,inplace = True)
    
    ## Discourse
    
    Discourse = pd.concat([discourse_Data_frame,Meta_data_df], axis = 1)
    Discourse.reset_index(drop = True,inplace = True)
    
    
    ## topos
    Topos = pd.concat([topos_Data_frame,Meta_data_df], axis = 1)
    Topos.reset_index(drop = True,inplace = True)
    
    return(pd.concat([Stageoflife,Events,Persona,Discourse,Topos],axis = 0))

### Looping through all the folders to get the data frames for each xml file

In [57]:
#Initializing an empty data frame
bess_df_all = pd.DataFrame()

### Looping through each of the collective biography
for a_files in os.listdir(path_overall):
    print(a_files)
    if(os.path.isdir(path_overall+"\\"+a_files)):
        
        ### Looping through each of the individual biography
        for bio_files in os.listdir(path_overall+"\\"+a_files):
            
            #print(os.path.isdir(path_overall+"\\"+a_files))
            if(os.path.isdir(path_overall+"\\"+a_files+"\\"+bio_files)):
                
                files = os.listdir(path_overall+"\\"+a_files+"\\"+bio_files)
                
                
                bess_files = [each_file for each_file in files if ".bess.xml" in each_file]
                ### Looping through each of the valid file in an individual biography
                for each_bess_file in bess_files:
                    #print(each_bess_file)
                    bess_file_path  = path_overall+"\\"+a_files+"\\"+bio_files+"\\"+each_bess_file
                    #print(each_bess_file)
                    bess_df_current = bess_to_df(bess_file_path)
                    bess_df_all = pd.concat([bess_df_all,bess_df_current],axis = 0)
                    
                    bess_df_all.reset_index(drop = True, inplace= True)

.DS_Store
a001
a007
a019A
a020
a034
a052A
a095A
a143
a160
a162
a185
a186
a207
a221
a221A
a270
a282
a302A
a319
a411
a468
a527
a557
a607
a624
a626
a628
a630
a664
a681
a712
a730
a739
a788
a789
a793
a813
a863
a887
a894
a920
a925
ACCOMPLISHMENT.txt
p067A
p091C
p131A
RCMCBWnewProject.xpr
TasteSearchProject.xpr


In [58]:
bess_df_all  = bess_df_all.drop_duplicates()

In [41]:
##### Mutual Information/  Non-negative matrix multiplication

In [61]:
bess_df_all.shape
bess_df_all.to_csv("CBW_Bess_tags.csv", index = False)

#### Examining Unique Classes under each category

#### Events

In [9]:
### We have a total of 1852 categories under Events
bess_df_all.Content[bess_df_all.Type == "Event"].value_counts().shape

(2400,)

In [15]:
## Looking at the Frequency
values = bess_df_all.Content[bess_df_all.Type == "Event"].value_counts()

In [24]:
counts = bess_df_all.Content[bess_df_all.Type == "Event"].value_counts()
100 * counts[counts <= 20].sum()/counts.sum()

6.833377908531693

#### Categories appearing just once

In [25]:
bess_df_all.drop_duplicates().shape

(166436, 10)

In [10]:
counts = bess_df_all.Content[bess_df_all.Type == "Event"].value_counts()
100 * counts[counts <= 20].sum()/counts.sum()

6.833377908531693

#### Persona Description

In [11]:
### We have a total of 1852 categories under Events
bess_df_all.Content[bess_df_all.Type == "personaDescription"].value_counts().shape

(236,)

In [26]:
## TO DO
## ASSOCIATION RULE STUDY FOR THESE

In [12]:
## Looking at the Frequency
bess_df_all.Content[bess_df_all.Type == "personaDescription"].value_counts()

beautiful                            542
dedicated or devoted                 389
energetic or untiring                336
brave                                330
happy                                281
intelligent                          273
loving                               239
sympathetic                          227
skilled, in occupation               217
self-sacrificing                     211
charming                             201
beloved                              193
kind                                 184
faith, confident in                  180
cheerful                             179
young                                179
devout                               177
noble in character                   176
willful or stubborn                  172
superior to other women              157
famous                               153
fearful                              147
physically weak or frail             146
charitable                           143
persistent      

In [13]:
## Different Segregation under Events
bess_df_all.Event.value_counts()

type                 114476
agentType             21339
name                  15807
locationSetting        5248
locationStructure      4430
dateSingle             3336
timeOfDay               737
season                  507
note                    444
dateRange               102
datesSingle               1
Name: Event, dtype: int64

In [14]:
counts = bess_df_all.Content[bess_df_all.Type == "personaDescription"].value_counts()
100 * counts[counts <= 20].sum()/counts.sum()

6.25147999052806

#### Stage of Life

In [15]:
### We have a total of 1852 categories under Events
bess_df_all.Content[bess_df_all.Type == "stageOfLife"].value_counts().shape

(14,)

In [16]:
## Looking at the Frequency
bess_df_all.Content[bess_df_all.Type == "stageOfLife"].value_counts()

middle                8541
culmination           2730
beginning             2404
end                   1033
after                  772
before                 228
\n                      53
 middle                 53
 beginning              15
 culmination            12
 after                  12
 end                     6
\n\t\t\t\t               4
 before                  1
Name: Content, dtype: int64

#### Discourse

In [17]:
### We have a total of 1852 categories under Events
bess_df_all.Content[bess_df_all.Type == "discourse"].value_counts().shape

(248,)

In [18]:
## Looking at the Frequency
bess_df_all.Content[bess_df_all.Type == "discourse"].value_counts()

evaluation                                                     2339
emphasis in typeface, punctuation                              2158
quotation, agent's speech, unique                              1671
present tense                                                  1670
description of agent or identified set of agents               1583
direct address, use of we                                      1535
focalization, attributing feelings                             1159
quotation, persona speech, unique                              1116
speech, reported not quoted                                    1087
digression or pause in life time                                993
quotation, unidentified                                         965
precepts                                                        864
quotation, persona writings about self                          829
dialogue                                                        811
description of environment                      

#### Topos

In [19]:
### We have a total of 1852 categories under Events
bess_df_all.Content[bess_df_all.Type == "topos"].value_counts().shape

(227,)

In [20]:
## Looking at the Frequency
bess_df_all.Content[bess_df_all.Type == "topos"].value_counts()

attachment or caring between the people and persona     546
fame in lifetime                                        495
dying or deathbed                                       493
association with great man or men                       460
hostility                                               438
recognition or tribute                                  427
triumph as fulfilled goal or ambition                   415
romance                                                 401
charity                                                 375
clothing, persona's                                     317
turning point                                           307
faith                                                   306
adversity                                               305
sacrifice or renunciation of self-interest or desire    303
memory                                                  296
failure as outcome, tragedy                             278
beauty as effect on others              

#### Author Analysis

In [21]:
bess_df_all.author.value_counts()

Unknown                                           27713
Albert Payson Terhune                             16407
Rosa Nouchette Carey                               9371
Ethel Colburn Mayne                                8977
A. J. Green Armytage                               8449
John Maw Darton                                    6365
Robert Cochrane                                    5924
Cameron Rogers                                     5747
Jeanie Douglas Cochrane                            5485
Frances A. Gerard                                  5320
Henry Ewart                                        4891
Millicent Garrett Fawcett and Steuart Beatrice     4515
George Barnett Smith                               4471
Duncan Aikman                                      3931
Louise Creighton                                   3480
Philip Walsingham Sergeant                         3102
Duncan Aikmen                                      2500
James Parton                                    

#### Saving the data

In [29]:
Author_list = bess_df_all.loc[:,["author","biographyID", "collectionID","personaName"]].drop_duplicates()
Author_list.shape

(321, 4)

In [30]:
Author_list.author.value_counts()

Unknown                                           39
Robert Cochrane                                   25
Ethel Colburn Mayne                               21
Albert Payson Terhune                             21
Millicent Garrett Fawcett and Steuart Beatrice    20
A. J. Green Armytage                              13
John Maw Darton                                   11
Louise Creighton                                  10
Henry Ewart                                       10
Rosa Nouchette Carey                              10
Jeanie Douglas Cochrane                           10
Myrtle Reed                                       10
Frances A. Gerard                                 10
Cameron Rogers                                     8
Duncan Aikman                                      7
George Barnett Smith                               6
Duncan Aikmen                                      5
Philip Walsingham Sergeant                         5
Clara H. Whitmore                             

In [36]:
Author_list[Author_list.personaName.isnull()]

,author,biographyID,collectionID,personaName
95866,None,bio05,a319,None
102101,"Hamilton Wright Mabie, Kate Stephens, eds.",bio04,a527,None


In [ ]:
# Scrape the data from the website
# Make a note of the biographios with no Persons
# Data Vis. for associations (heat map)
# (word features) - TF-IDF 


## 1. who is being remmebered? (Excluding writers from a different race - African america) - selection bias shuoldn't arrise- 
We are making decision about what is being remember about the best - cultural memory -
Understanding power structures
### 2. Does this project help or hinder portraying the memory of some particular woman

### 3. DS and shcholarship - humanities vs. engineering - eng. would be happy to have ai to do somehting - what we do vs. the scholar
# do we replace them or ccomplement them - 
# Asnwering about the relationship about analytics and scholarship


### Alison - middle range reading - (distant reading) - digital methods to read the whole corpus - solution: take the best of both
#the world - ## closed reading vs. distant reading - #Talk about human in the loop (DS can't do everything)

